In [8]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
import csv
from statistics import mean, median,variance,stdev
from datetime import datetime
import glob, re
from sklearn import *

# Air data
air_reserve = pd.read_csv('D:/MachineLearning/ML_Assignment/all_data/air_reserve.csv')
air_visit_data = pd.read_csv('D:/MachineLearning/ML_Assignment/all_data/air_visit_data.csv')
air_store_info = pd.read_csv('D:/MachineLearning/ML_Assignment/all_data/air_store_info.csv')
# HPG data
hpg_reserve =  pd.read_csv('D:/MachineLearning/ML_Assignment/all_data/hpg_reserve.csv')
hpg_store_info = pd.read_csv('D:/MachineLearning/ML_Assignment/all_data/hpg_store_info.csv')
# Diverse data
store_id_relation = pd.read_csv('D:/MachineLearning/ML_Assignment/all_data/store_id_relation.csv')
date_info = pd.read_csv('D:/MachineLearning/ML_Assignment/all_data/date_info.csv')
# Submission file
sample_submission = pd.read_csv('D:/MachineLearning/ML_Assignment/all_data/sample_submission.csv')
print("Start")
# merge the tables to add any missing datas in both dataframes based on store_id_relation
hpg_reserve = pd.merge(hpg_reserve, store_id_relation, how='left', on=['hpg_store_id'])
air_reserve = pd.merge(air_reserve, store_id_relation, how='left', on=['air_store_id'])
# Formated coloum of date year month
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['visit_datetime'] = hpg_reserve['visit_datetime'].dt.date
# Formated coloum of date year month
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['visit_datetime'] = air_reserve['visit_datetime'].dt.date
# Formated coloum of date year month
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])
hpg_reserve['reserve_datetime'] = hpg_reserve['reserve_datetime'].dt.date
# Formated coloum of date year month
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])
air_reserve['reserve_datetime'] = air_reserve['reserve_datetime'].dt.date
# Formated coloum of date year month
air_visit_data['visit_date'] = pd.to_datetime(air_visit_data['visit_date'])
air_visit_data['visit_date'] = air_visit_data['visit_date'].dt.date
air_visit_data = air_visit_data.rename(columns={'visit_date':'visit_datetime'})
# Formated coloum of date year month
date_info['calendar_date'] = pd.to_datetime(date_info['calendar_date'])
date_info['calendar_date'] = date_info['calendar_date'].dt.date
date_info = date_info.rename(columns={'calendar_date':'visit_datetime'})
# Formated coloum of date year month
sample_submission['air_store_id'] = sample_submission['id'].map(lambda x: '_'.join(x.split('_')[:2]))
sample_submission['visit_datetime'] = sample_submission['id'].map(lambda x: str(x).split('_')[2])
sample_submission['visit_datetime'] = pd.to_datetime(sample_submission['visit_datetime'])
sample_submission['visit_datetime'] = sample_submission['visit_datetime'].dt.date

df_ah_re = pd.concat([air_reserve,hpg_reserve],ignore_index = True)
df_ah_re = df_ah_re.fillna(0)

df_ah_re = df_ah_re.groupby(['air_store_id','hpg_store_id','visit_datetime'])['reserve_visitors'].sum().reset_index()

df_ah_re = pd.merge(df_ah_re, date_info, how = 'left', on=['visit_datetime'])
df_ah_re = pd.merge(df_ah_re, air_store_info, how = 'left', on = ['air_store_id'])
df_ah_re = pd.merge(df_ah_re, hpg_store_info, how = 'left', on = ['hpg_store_id'])
df_ah_re = df_ah_re.fillna(0)

sample_submission = pd.merge(sample_submission,date_info, how = 'left', on = ['visit_datetime'])
sample_submission = pd.merge(sample_submission,df_ah_re, how = 'left', on = ['air_store_id','visit_datetime','day_of_week','holiday_flg'])

df_ah_re.air_genre_name = df_ah_re.air_genre_name.astype('category')
df_ah_re.hpg_genre_name = df_ah_re.hpg_genre_name.astype('category')
df_ah_re.day_of_week = df_ah_re.day_of_week.astype('category')
df_ah_re.reserve_visitors = df_ah_re.reserve_visitors.astype('float64')

df_ah_re.air_genre_name = df_ah_re.air_genre_name.cat.codes
df_ah_re.hpg_genre_name = df_ah_re.hpg_genre_name.cat.codes
df_ah_re.day_of_week = df_ah_re.day_of_week.cat.codes

sample_submission = sample_submission.fillna(0)
sample_submission.air_genre_name = sample_submission.air_genre_name.astype('category')
sample_submission.hpg_genre_name = sample_submission.hpg_genre_name.astype('category')
sample_submission.day_of_week =sample_submission.day_of_week.astype('category')
sample_submission.air_genre_name = sample_submission.air_genre_name.cat.codes
sample_submission.hpg_genre_name = sample_submission.hpg_genre_name.cat.codes
sample_submission.day_of_week = sample_submission.day_of_week.cat.codes

sample_test = sample_submission.drop(['id','air_store_id','hpg_store_id','visit_datetime','air_area_name','latitude_x','longitude_x','hpg_area_name','latitude_y','longitude_y','visitors','reserve_visitors','air_genre_name','hpg_genre_name'], axis=1)
sample_test = sample_test.reset_index(drop=True)
print("sample_test")
print(sample_test)
df_ah_re_test = df_ah_re.drop(['air_store_id','hpg_store_id','visit_datetime','air_area_name','latitude_x','longitude_x','hpg_area_name','latitude_y','longitude_y','reserve_visitors','air_genre_name','hpg_genre_name'], axis=1)
df_ah_re_test = df_ah_re_test.reset_index(drop=True)
print("df_ah_re_test")
print(df_ah_re_test)
print( df_ah_re.reserve_visitors)
xs = df_ah_re_test
y = df_ah_re.reserve_visitors
regressor = LinearRegression()
regressor.fit(xs,y)
print(df_ah_re.isnull().sum())
print(sample_test.isnull().sum())
reserve_pre = regressor.predict(sample_test)
reserve_pre = pd.Series(reserve_pre)
submit = pd.concat([sample_submission.id, reserve_pre], axis=1)
submit.columns = ['id','visitors']
print(submit)
submit.to_csv('submit.csv', index=False)
print("Done")


Start


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



sample_test
       day_of_week  holiday_flg
0                3            0
1                1            0
2                5            0
3                6            0
4                4            0
5                0            0
6                2            1
7                3            0
8                1            0
9                5            0
10               6            1
11               4            1
12               0            1
13               2            0
14               3            0
15               1            0
16               5            0
17               6            0
18               4            0
19               0            0
20               2            0
21               3            0
22               1            0
23               5            0
24               6            0
25               4            0
26               0            0
27               2            0
28               3            0
29               1          